In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/l2sp' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes)
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 5000] LR: 0.010000
1/7 | Total:0:00:03 | ETA:0:00:20 | Loss:4.8989410400390625 | top1:43.75
7/7 | Total:0:00:06 | ETA:0:00:02 | Loss:2.401725093523661 | top1:48.958335876464844


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


107/107 | Total:0:00:50 | ETA:0:00:00 | Loss:1.1856436729431152 | top1:34.959503173828125
26/26 | Total:0:00:34 | ETA:0:00:00 | Loss:0.6743796009283799 | top1:68.5897445678711

Epoch: [2 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:1.008162260055542 | top1:53.125
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.9121402502059937 | top1:45.3125

Epoch: [3 | 5000] LR: 0.024000
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.6931575536727905 | top1:59.375
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7648655275503794 | top1:52.60416793823242

Epoch: [4 | 5000] LR: 0.031000
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.7936456799507141 | top1:40.625
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7320666909217834 | top1:46.875

Epoch: [5 | 5000] LR: 0.038000
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.7103613018989563 | top1:56.25
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7226517498493195 | top1:57.29166793823242

Epoch: [6 | 5000] LR: 0.045000
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.67523598

1/7 | Total:0:00:02 | ETA:0:00:14 | Loss:0.570224404335022 | top1:90.625
7/7 | Total:0:00:06 | ETA:0:00:01 | Loss:0.5351451834042867 | top1:87.5

Epoch: [46 | 5000] LR: 0.079991
1/7 | Total:0:00:02 | ETA:0:00:15 | Loss:0.4623572826385498 | top1:87.5
7/7 | Total:0:00:06 | ETA:0:00:02 | Loss:0.5042112171649933 | top1:88.02083587646484

Epoch: [47 | 5000] LR: 0.079990
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.8393118381500244 | top1:68.75
7/7 | Total:0:00:05 | ETA:0:00:01 | Loss:0.8126531740029653 | top1:75.52083587646484

Epoch: [48 | 5000] LR: 0.079990
1/7 | Total:0:00:02 | ETA:0:00:14 | Loss:0.694442629814148 | top1:75.0
7/7 | Total:0:00:06 | ETA:0:00:01 | Loss:0.6502487361431122 | top1:83.33333587646484

Epoch: [49 | 5000] LR: 0.079989
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.4184896945953369 | top1:96.875
7/7 | Total:0:00:05 | ETA:0:00:01 | Loss:0.6857090989748637 | top1:79.6875

Epoch: [50 | 5000] LR: 0.079989
1/7 | Total:0:00:02 | ETA:0:00:15 | Loss:0.5984528064727783 | top1:81.

1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.5942540764808655 | top1:78.125
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.6536052624384562 | top1:83.33333587646484

Epoch: [89 | 5000] LR: 0.079953
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.501054584980011 | top1:93.75
7/7 | Total:0:00:05 | ETA:0:00:01 | Loss:0.576175332069397 | top1:90.10417175292969

Epoch: [90 | 5000] LR: 0.079952
1/7 | Total:0:00:02 | ETA:0:00:14 | Loss:0.5336644649505615 | top1:87.5
7/7 | Total:0:00:07 | ETA:0:00:02 | Loss:0.5373051961263021 | top1:88.02083587646484

Epoch: [91 | 5000] LR: 0.079951
1/7 | Total:0:00:02 | ETA:0:00:15 | Loss:0.7813801765441895 | top1:75.0
7/7 | Total:0:00:05 | ETA:0:00:01 | Loss:0.6384987930456797 | top1:83.33333587646484

Epoch: [92 | 5000] LR: 0.079949
1/7 | Total:0:00:01 | ETA:0:00:12 | Loss:0.600726842880249 | top1:90.625
7/7 | Total:0:00:05 | ETA:0:00:01 | Loss:0.5567969779173533 | top1:87.5

Epoch: [93 | 5000] LR: 0.079948
1/7 | Total:0:00:02 | ETA:0:00:15 | Loss:0.6604927778244019 |

...

7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2247986520330111 | top1:97.39583587646484

Epoch: [4815 | 5000] LR: 0.000003
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.17217713594436646 | top1:100.0
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.1949337919553121 | top1:98.95833587646484

Epoch: [4816 | 5000] LR: 0.000003
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.24103832244873047 | top1:96.875
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.22342480719089508 | top1:97.91667175292969

Epoch: [4817 | 5000] LR: 0.000003
1/7 | Total:0:00:02 | ETA:0:00:13 | Loss:0.2103041112422943 | top1:96.875
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.2167816311120987 | top1:98.4375

Epoch: [4818 | 5000] LR: 0.000003
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.2581322193145752 | top1:96.875
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.22663971285025278 | top1:97.91667175292969

Epoch: [4819 | 5000] LR: 0.000003
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.24531373381614685 | top1:96.875
7/7 | Total:0:00:05 | ETA:0:00:01 | 

7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.21416877955198288 | top1:98.4375

Epoch: [4857 | 5000] LR: 0.000002
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.1699535995721817 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.24559730291366577 | top1:96.875

Epoch: [4858 | 5000] LR: 0.000002
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.2041744589805603 | top1:96.875
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.22704758743445078 | top1:97.91667175292969

Epoch: [4859 | 5000] LR: 0.000002
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.23903246223926544 | top1:93.75
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.2539789130290349 | top1:97.39583587646484

Epoch: [4860 | 5000] LR: 0.000002
1/7 | Total:0:00:02 | ETA:0:00:13 | Loss:0.2336212545633316 | top1:96.875
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.25375596433877945 | top1:96.875

Epoch: [4861 | 5000] LR: 0.000002
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.18015506863594055 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2274335250258445

1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.1723797619342804 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.20967700084050497 | top1:98.4375

Epoch: [4901 | 5000] LR: 0.000001
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.32385510206222534 | top1:93.75
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.2523769562443097 | top1:97.39583587646484
107/107 | Total:0:00:46 | ETA:0:00:00 | Loss:0.6291074451999129 | top1:85.55140686035156
26/26 | Total:0:00:25 | ETA:0:00:00 | Loss:1.595769932636848 | top1:67.84615325927734

Epoch: [4902 | 5000] LR: 0.000001
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.2015775740146637 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.21850203474362692 | top1:98.95833587646484

Epoch: [4903 | 5000] LR: 0.000001
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.1743219792842865 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2145779033501943 | top1:96.875

Epoch: [4904 | 5000] LR: 0.000001
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.21082720160484314 | top1:100

7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.20409491658210754 | top1:98.95833587646484

Epoch: [4943 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.20492085814476013 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2209387868642807 | top1:97.91667175292969

Epoch: [4944 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.16648665070533752 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.22311468670765558 | top1:97.91667175292969

Epoch: [4945 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.1898595243692398 | top1:96.875
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.19279091556866965 | top1:98.95833587646484

Epoch: [4946 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.2589767277240753 | top1:93.75
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.20399662107229233 | top1:97.91667175292969

Epoch: [4947 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.2222936749458313 | top1:96.875
7/7 | Total:0:00:03 | ETA:0

7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.22844119866689047 | top1:97.39583587646484

Epoch: [4985 | 5000] LR: 0.000000
1/7 | Total:0:00:02 | ETA:0:00:13 | Loss:0.3676789104938507 | top1:96.875
7/7 | Total:0:00:05 | ETA:0:00:01 | Loss:0.27094799776872 | top1:97.39583587646484

Epoch: [4986 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.3422446846961975 | top1:90.625
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.21907219290733337 | top1:97.91667175292969

Epoch: [4987 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.2106979638338089 | top1:100.0
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.2004681999484698 | top1:97.91667175292969

Epoch: [4988 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:12 | Loss:0.2979273200035095 | top1:96.875
7/7 | Total:0:00:04 | ETA:0:00:01 | Loss:0.2427881956100464 | top1:96.35417175292969

Epoch: [4989 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.16854169964790344 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00: